# <----------------------Data Collection-------------------------->

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("gunavenkatdoddi/eye-diseases-classification")

print("Path to dataset files:", path)

100%|██████████| 736M/736M [00:06<00:00, 113MB/s]


Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/gunavenkatdoddi/eye-diseases-classification/versions/1


# <------------------importing the Libraries---------------------------------->

In [2]:
import numpy as np
import pathlib
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
# Import preprocess_input from the specific model or imagenet_utils
from tensorflow.keras.applications.vgg19 import preprocess_input
# For VGG19
from tensorflow.keras.applications.vgg19 import VGG19
#from tensorflow.keras.applications.imagenet_utils import preprocess_input # For general ImageNet models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dropout, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import plot_model
from sklearn.metrics import classification_report, confusion_matrix

In [4]:
batch_size = 64

# <-------------------------------------------------Creating Training and testing data -------------------->

In [11]:
import os
import shutil
import random

# Define paths
dataset_dir = "//root/.cache/kagglehub/datasets/gunavenkatdoddi/eye-diseases-classification/versions/1/dataset"  # Change this to your actual dataset path
train_dir = "train_data"
test_dir = "test_data"

# Define train-test split ratio
split_ratio = 0.2  # 20% test, 80% train

# Create train and test folders
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Iterate over each class
for class_name in os.listdir(dataset_dir):
    class_path = os.path.join(dataset_dir, class_name)
    if not os.path.isdir(class_path):
        continue

    # Create class subfolders inside train & test
    os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
    os.makedirs(os.path.join(test_dir, class_name), exist_ok=True)

    # Get all image files
    images = [img for img in os.listdir(class_path) if img.endswith(('.jpg', '.png', '.jpeg'))]
    random.shuffle(images)  # Shuffle for randomness

    # Split into train and test sets
    split_idx = int(len(images) * (1 - split_ratio))
    train_images = images[:split_idx]
    test_images = images[split_idx:]

    # Move images to respective folders
    for img in train_images:
        shutil.move(os.path.join(class_path, img), os.path.join(train_dir, class_name, img))

    for img in test_images:
        shutil.move(os.path.join(class_path, img), os.path.join(test_dir, class_name, img))

print("✅ Data successfully split into training and testing folders!")


✅ Data successfully split into training and testing folders!


# <----------------------------------ImageDataGenerator on training and test data-------------------->

In [29]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg19 import preprocess_input

# Train data generator (with augmentations)
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    brightness_range=[0.8, 1.2],
    vertical_flip=True,
    horizontal_flip=True
)

# Test data generator (only preprocessing, no augmentations)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# Load Train Data
train_data = train_datagen.flow_from_directory(
    "/content/train_data",
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

# Load Test Data
test_data = test_datagen.flow_from_directory(
    "/content/test_data",
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

print("✅ Train & Test Data Successfully Loaded!")


Found 3372 images belonging to 4 classes.
Found 845 images belonging to 4 classes.
✅ Train & Test Data Successfully Loaded!


# <--------    Pretrained CNN model Vgg19----------------------->





In [30]:
import tensorflow as tf
from tensorflow.keras.applications import VGG19  # You can also use ResNet50, MobileNetV2, etc.
from tensorflow.keras.layers import Flatten, Dense, Dropout

# Load Pretrained Model (Feature Extractor)
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze All Convolutional Layers (So they don’t get updated)
for layer in base_model.layers:
    layer.trainable = False


#<----------Adding Dense Layer-------------->

In [31]:
from tensorflow.keras.models import Model
# Add Custom Fully Connected Layers
x = Flatten()(base_model.output)  # Convert feature maps into 1D vector
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)  # Prevents overfitting
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
outputs = Dense(4, activation='softmax')(x)  # Change '4' to match the number of classes & assign to outputs

# Define the Model
model = Model(inputs=base_model.input, outputs=outputs) # Pass outputs to the Model function

# <-----------Configure the learning process : Compilation------>

In [32]:
# Compile the Model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Model Summary
model.summary()


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)           │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 224, 224, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 224, 224, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 112, 112, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 112, 112, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 56, 56, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv4 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 28, 28, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 28, 28, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv4 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 14, 14, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv4 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 33,002,308 (125.89 MB)

 Trainable params: 12,977,924 (49.51 MB)

 Non-trainable params: 20,024,384 (76.39 MB)

#<------------------Training the model-------------->

In [33]:
# Train the Model
history = model.fit(
    train_data,   # Training dataset
    validation_data=test_data,  # Validation dataset
    epochs=50,  # Number of epochs
    steps_per_epoch=len(train_data),
    validation_steps=len(test_data)

)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 86s 2s/step - accuracy: 0.4625 - loss: 12.3447 - val_accuracy: 0.6355 - val_loss: 1.7184
Epoch 2/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 136s 1s/step - accuracy: 0.5770 - loss: 2.1926 - val_accuracy: 0.5669 - val_loss: 1.2422
Epoch 3/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 84s 2s/step - accuracy: 0.5640 - loss: 1.1182 - val_accuracy: 0.5657 - val_loss: 0.9867
Epoch 4/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 83s 2s/step - accuracy: 0.5588 - loss: 1.0221 - val_accuracy: 0.5692 - val_loss: 0.9773
Epoch 5/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 82s 2s/step - accuracy: 0.5694 - loss: 1.0277 - val_accuracy: 0.5775 - val_loss: 0.9151
Epoch 6/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 80s 1s/step - accuracy: 0.5632 - loss: 1.0373 - val_accuracy: 0.5598 - val_loss: 0.9871
Epoch 7/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 76s 1s/step - accuracy: 0.5769 - loss: 0.9590 - val_accuracy: 0.5799 - val_loss: 0.9075
Epoch 8/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 84s 1s/step - accuracy: 0.5700 - loss: 0.9702 - val_accuracy: 0.5172 - val_los

In [34]:
model.save('eye_vgg19.h5')

In [35]:
model = tf.keras.models.load_model("eye_vgg19.h5")

In [36]:
class_names = ['cataract', 'diabetic_retinopathy', 'glaucoma', 'normal']


In [37]:
from tensorflow.keras.preprocessing import image
def eye_diseaes(image_path):
    img = tf.keras.utils.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    # Make predictions
    predictions = model.predict(img_array)

    # Get the index of the class with the highest predicted probability
    predicted_index = np.argmax(predictions, axis=1)[0]
    predicted_class = class_names[predicted_index]
    confidence = predictions[0][predicted_index]

    return predicted_class, confidence

In [39]:
image_path = "/content/train_data/glaucoma/1209_left.jpg"
predicted_class, confidence = eye_diseaes(image_path)
print("Predicted Disease:", predicted_class)
print("Confidence:", confidence)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Predicted Disease: glaucoma
Confidence: 0.747985


In [40]:
image_path = "/content/train_data/normal/1034_right.jpg"
predicted_class, confidence = eye_diseaes(image_path)
print("Predicted Disease:", predicted_class)
print("Confidence:", confidence)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
Predicted Disease: normal
Confidence: 0.42947558
